Capstone 1 Wrangling

All companies that participate in wholesale energy markets are required to submit a quarterly report detailing each transaction to the Federal Energy Regulatory Commission (FERC). This information is then made publicly available for download on FERC’s website at https://eqrreportviewer.ferc.gov/. The website allows you to download a zipped file that contains all of the submissions made for each quarter. Each submission is kept in a zipped folder and can contain up to 4 csv files:

1.	Identity table. Required to be submitted by all entities. Contains information about the entity that is filing the report.
2.	Index table. Required to be submitted by all entities, but can be left blank if nothing to be reported. Lists the index price publishers to which transactions have been reported.
3.	Contracts table. Only filed if the entity has something to report. Lists any all of the currently active wholesale contracts where the entity is the seller of energy.
4.	Transactions table. Only filed if the entity has something to report. Lists all of the sales of energy made by the entity.

Once downloaded the first step is to unzip each quarterly filing and each individual submission folder. Since thousands of folders needed to be unzipped I loaded all of the zipped quarterly filings into a single directory and then used the below to programmatically unzip and move each one to a new directory.


In [ ]:
import zipfile
import os 
import fnmatch as fn
import pandas as pd
import time

#start = time.time()

root = r"C:\Users\anhem44\Desktop\Capstone1\Zip"
pattern = '*.zip'

for root, dirs, files in os.walk(rootPath):
    for filename in fnmatch.filter(files, pattern):
        zipfile.ZipFile(os.path.join(root, filename)).extractall(os.path.join(r"C:\Users\anhem44\Desktop\Capstone1\unzip1", os.path.splitext(filename)[0]))

root = r"C:\Users\anhem44\Desktop\Capstone1\unzip1"

for root, dirs, files in os.walk(rootPath):
    for filename in fnmatch.filter(files, pattern):
        zipfile.ZipFile(os.path.join(root, filename)).extractall(os.path.join(r"C:\Users\anhem44\Desktop\Capstone1\unzip2", os.path.splitext(filename)[0]))\



At this point I was left with a directory filled with thousands of unzipped folders, each of which contained between 2 and 4 .csv files. For the purposes of my project I only need the transactions table (#4 above) so I wrote a the belpw scropt which looped through each folder, pulled out only the transactions table (this was straightforward as entities are required to submit these files as entityname_transactions.csv), and then concatenated each .csv file to a pandas data frame.

In [ ]:
root = r"C:\Users\anhem44\Desktop\Capstone1\unzip2"
pattern = "*transactions.csv"
combined_csv = pd.DataFrame()

dtypes = {'transaction_unique_id':'str',
 'seller_company_name':'str',
 'customer_company_name':'str',
 'ferc_tariff_reference':'str',
 'contract_service_agreement':'str',
 'transaction_unique_identifier':'str',
 'transaction_begin_date':'str',
 'transaction_end_date':'str',
 'trade_date':'str',
 'exchange_brokerage_service':'str',
 'type_of_rate':'str',
 'time_zone':'str',
 'point_of_delivery_balancing_authority':'str',
 'point_of_delivery_specific_location':'str',
 'class_name':'str',
 'term_name':'str',
 'increment_name':'str',
 'increment_peaking_name':'str',
 'product_name':'str',
 'transaction_quantity':'float',
 'price':'float',
 'rate_units':'str',
 'standardized_quantity':'float',
 'standardized_price':'float',
 'total_transmission_charge':'float',
 'total_transaction_charge':'float'}

for path, subdirs, files in os.walk(root):
    filenames = [] 
    for name in files:
        try :
            if fn.fnmatch(name, pattern):
                filenames.append(os.path.join(path, name))
        except:
            pass


for f in filenames :
    try:
        combined_csv = pd.concat([combined_csv, pd.read_csv(f,dtype=dtypes,parse_dates=["trade_date"])])
    except:
        pass
    


In the transactions table transaction begin date and transaction end date were stored as integers (e.g. 20160716120000 for 2016-07-16 12:00:00) so I used the following to convert these integers into the appropriate date time.

In [ ]:
combined_csv['transaction_begin_date'] = pd.to_datetime(combined_csv['transaction_begin_date'])      
combined_csv['transaction_end_date'] = pd.to_datetime(combined_csv['transaction_end_date'])  

Finally, since the goal of this project was to predict prices for a FERC specific time period and season I used the following to create these variables in the data frame.

In [ ]:
combined_csv['FERC_time'] = 'off_peak'
combined_csv['FERC_time'][(combined_csv['transaction_begin_date'].dt.weekday <= 5) & (combined_csv['transaction_begin_date'].dt.hour >= 6) & (combined_csv['transaction_begin_date'].dt.hour <= 21)]='peak' 

combined_csv['FERC_season'] = 'shoulder'
combined_csv['FERC_season'][(combined_csv['transaction_begin_date'].dt.month <= 2) | (combined_csv['transaction_begin_date'].dt.month >= 12)]='winter' 
combined_csv['FERC_season'][(combined_csv['transaction_begin_date'].dt.month <= 8) & (combined_csv['transaction_begin_date'].dt.month >= 6)]= 'summer'

For simplicities sake, I've exported this data frame to a csv so that I don't have to clean the data every time.

In [ ]:
combined_csv.to_csv(r"C:\Users\anhem44\Desktop\Test\combined_csv.csv")
#end = time.time()
#print(end - start)